In [652]:
from __future__ import print_function
# key libs
import numpy as np
import re
import nltk
import pandas as pd
import glob
import codecs

# nlp libs
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 

# processing
from sklearn.model_selection import train_test_split

# LDA
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.decomposition import LatentDirichletAllocation
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF


# bring in my pickled vectorizers
import pickle
import dill

from nltk import SnowballStemmer
from nltk import PorterStemmer
from nltk import LancasterStemmer
from nltk.stem import WordNetLemmatizer

In [653]:
RSEED = 0
bow = 'tf' # tf,tfid
stem_type = 'lemma' # snow, lemma
n_gram = '1gm' # 1gm or 2gm
topic_model = 'lda' #lda, nmf

In [654]:
file_link = '../data/samples/fifty_shades.txt'

### 1. PIPELINE 1 - vectorize and topic model 

In [655]:
# open dill and wordnet_lemmatizer / snowball
vectorizer = dill.load(open('../data/vectors/'+bow+'_vectorizer_'+ stem_type + '_' + str(n_gram), 'rb'))
wordnet_lemmatizer = WordNetLemmatizer()
snow = SnowballStemmer('english')

In [656]:
vector = vectorizer.transform([file_link])

In [657]:
model = dill.load(open('../data/vectors/'+ topic_model +'_'+ stem_type + '_' + n_gram,'rb'))
topic_vector = model.transform(vector)

In [658]:
# place in panda for easy manipulation
df_excerpt_a = pd.DataFrame(topic_vector, columns=['topic_'+ str(i)for i in range(1,21)])
df_excerpt_a

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20
0,0.000397,0.000397,0.000397,0.192501,0.000397,0.000397,0.000397,0.000397,0.52183,0.000397,0.000397,0.000397,0.000397,0.000397,0.033007,0.000397,0.000397,0.000397,0.099826,0.146884


### 2. PIPELINE 2 - sentiment analysis and word counts

In [659]:
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
file=open(file_link)
t=file.read()
book_excerpt = TextBlob(t)
word_count = len(book_excerpt.words)
sentence_count =len(book_excerpt.sentences)
avg_len = word_count/sentence_count
sentiment_excerpt = [[word_count,sentence_count,avg_len,book_excerpt.sentiment[0],book_excerpt.sentiment[1]]]

In [660]:
sentiment_excerpt

[[590, 47, 12.553191489361701, 0.007202380952380952, 0.4518614718614719]]

In [661]:
df_excerpt_b = pd.DataFrame(sentiment_excerpt, \
                            columns = ['word_count','sentence_count','sentence_length','polarity','subjectivity'])

In [662]:
df_excerpt_b

,word_count,sentence_count,sentence_length,polarity,subjectivity
0,590,47,12.553191,0.007202,0.451861


In [663]:
# join 

In [664]:
df_excerpt = pd.concat([df_excerpt_a,df_excerpt_b], axis=1)

In [665]:
df_excerpt

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,...,topic_16,topic_17,topic_18,topic_19,topic_20,word_count,sentence_count,sentence_length,polarity,subjectivity
0,0.000397,0.000397,0.000397,0.192501,0.000397,0.000397,0.000397,0.000397,0.52183,0.000397,...,0.000397,0.000397,0.000397,0.099826,0.146884,590,47,12.553191,0.007202,0.451861


In [666]:
# tranform
from sklearn import preprocessing



# log transform the counts 
column_names_to_log_1 = ['word_count', 'sentence_count', 'sentence_length']

df_excerpt.loc[:,column_names_to_log_1] = df_excerpt.loc[:,column_names_to_log_1].apply(np.log)


# normalize subjectivity and polarity
column_names_to_normalize = ['subjectivity', 'polarity']

# load the scaler 
min_max_scaler = dill.load(open('../data/vectors/scaler','rb'))

x = df_excerpt[column_names_to_normalize].values
x_scaled = min_max_scaler.transform(x) # only transform
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = df_excerpt.index)
df_excerpt[column_names_to_normalize] = df_temp

# log transform topics
df_excerpt.loc[:,'topic_1':'topic_20'] = df_excerpt.loc[:,'topic_1':'topic_20'].apply(np.log)

In [667]:
df_excerpt

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,...,topic_16,topic_17,topic_18,topic_19,topic_20,word_count,sentence_count,sentence_length,polarity,subjectivity
0,-7.832014,-7.832014,-7.832014,-1.647655,-7.832014,-7.832014,-7.832014,-7.832014,-0.650413,-7.832014,...,-7.832014,-7.832014,-7.832014,-2.304325,-1.918114,6.380123,3.850148,2.529975,0.227907,0.393448


In [668]:
# load the corpus vector from train
corpus = pd.read_csv('../data/final_train.csv')
corpus = corpus.drop(columns ='Unnamed: 0')
corpus.head()

,book_title,author_name,book_location,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,...,topic_16,topic_17,topic_18,topic_19,topic_20,word_count,sentence_count,sentence_length,polarity,subjectivity
0,Behind the Beyond,Stephen Leacock,../data/gutenberg/Stephen Leacock___Behind the...,-12.002609,-12.002609,-12.002609,-2.871504,-3.153252,-12.002609,-1.804827,...,-2.601882,-12.002609,-2.533365,-12.002609,-1.715871,10.312314,7.719130,2.593184,0.463344,0.438980
1,Tommy and Co,Jerome Klapka Jerome,../data/gutenberg/Jerome Klapka Jerome___Tommy...,-12.689609,-3.822413,-12.689609,-12.689609,-12.689609,-12.689609,-3.369898,...,-2.813341,-1.759709,-12.689609,-12.689609,-2.188462,10.936636,8.395477,2.541159,0.481264,0.534175
2,Winsome Winnie and other New Nonsense Novels,Stephen Leacock,../data/gutenberg/Stephen Leacock___Winsome Wi...,-12.318240,-4.697728,-2.575599,-2.319330,-4.822360,-2.884098,-1.610666,...,-2.252721,-3.056927,-12.318240,-12.318240,-2.031256,10.587266,8.020270,2.566996,0.506857,0.488050
3,The Moccasin Ranch,Hamlin Garland,../data/gutenberg/Hamlin Garland___The Moccasi...,-11.631952,-11.631952,-0.884569,-1.790058,-4.037696,-11.631952,-3.698611,...,-11.631952,-11.631952,-11.631952,-11.631952,-4.415382,9.817221,7.224753,2.592468,0.397930,0.594432
4,Three Ghost Stories,Charles Dickens,../data/gutenberg/Charles Dickens___Three Ghos...,-5.966696,-2.489925,-3.869492,-2.632903,-11.630886,-11.630886,-0.820747,...,-3.382406,-4.625073,-4.409436,-4.862522,-2.233253,9.960860,6.907755,3.053105,0.404810,0.465239


In [669]:
# columns to drop before finding similarity
drop_cols =['word_count','sentence_count']
corpus = corpus.drop(columns =drop_cols)
df_excerpt = df_excerpt.drop(columns = drop_cols)

In [670]:
print(corpus.shape)
print(df_excerpt.shape)

(2428, 26)
(1, 23)


In [671]:
# shape em up to numpy arrays
given_excerpt = np.array(df_excerpt)
search_in = np.array(corpus.iloc[:,3:])

In [672]:
print(given_excerpt.shape)
print(search_in.shape)

(1, 23)
(2428, 23)


In [673]:
from sklearn.metrics.pairwise import cosine_similarity
results = cosine_similarity(search_in, given_excerpt)

In [674]:
show_me = pd.DataFrame(results).sort_values(0, ascending=False).head(5)
show_me

,0
634,0.973251
2302,0.961484
1650,0.961472
2167,0.961386
999,0.961275


In [675]:
title = corpus.iloc[list(show_me.index),0:2].values
print(title)
similar_book = title[0][0] + ' by '+ title[0][1]
print(similar_book)

[['Anna Christie' 'Eugene O']
 ["Bill's Paper Chase, Lady of the Barge and Others, Part 3"
  'William Wymark Jacobs']
 ['The Persecution of Bob Pretty, Odd Craft, Part 9'
  'William Wymark Jacobs']
 ["Manners Makyth Man, Ship's Company, Part 12" 'William Wymark Jacobs']
 ['Dirty Work, Deep Waters, Part 11' 'William Wymark Jacobs']]
Anna Christie by Eugene O


50 shades of gray<br>
Anna Christie	by Eugene O	<br>
Anna Christie is the story of a former prostitute who falls in love, but runs into difficulty in turning her life around.

The Blue Djinn of Babylon<br>
The Parasite	Sir Arthur Conan Doyle	<br>
The Parasite makes use of a form of mind control similar to the mesmerism of the Victorian era; it works on some hosts but not others.
